In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import json
import os

In [19]:
root_path = os.path.join(os.environ['HOME'], 'projects/tuplex-public/tuplex/cmake-build-debug/dist/bin')
path = root_path + '/aws_job.json'

In [21]:
os.path.isfile(path)

True

In [25]:
with open(path, 'r') as fp:
    data = json.load(fp)

In [26]:
data

{'stageStartTimestamp': 391858287253828,
 'tasks': [{'container': {'reused': False,
    'requestId': 'e4cf5f8d-27c6-4051-aa13-cc66d10c830b',
    'uuid': 'de711321-fc87-4798-9bc0-7d446a7a71c1',
    'msRemaining': 595898,
    'requestsServed': 1,
    'startTimestamp': 1643339402105915345,
    'deadlineTimestamp': 1643339406233805553},
   'invoked_containers': [{'reused': False,
     'requestId': '876aa79f-0037-4ea6-be37-6b4e0ad60869',
     'uuid': '2eb8ca6f-45bb-48bc-8985-1e78e551b43a',
     'msRemaining': 598170,
     'requestsServed': 1,
     'startTimestamp': 1643339404011208154,
     'deadlineTimestamp': 1643339405895508969},
    {'reused': False,
     'requestId': '582ccc33-8d18-46c9-8212-69b25fedf51c',
     'uuid': '655d9e80-2c34-4463-9a0d-9bf70afbaef1',
     'msRemaining': 598597,
     'requestsServed': 1,
     'startTimestamp': 1643339403956466641,
     'deadlineTimestamp': 1643339405384964291},
    {'reused': False,
     'requestId': 'adddd99c-9e0b-4080-9e4b-4bfe82233771',
     

In [27]:
import datetime

In [ ]:
stageStart